# The Battle of the Neighborhoods

## Import neccesary Libraries

In [76]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'W0AZNSPW3WX4IL51ARTGAFF2LDSGP42IEHW3EP4CVXYBLCY0' # your Foursquare ID
CLIENT_SECRET = 'PFQMJMGNLQWSXH0SN0ENIQSPO0URKN25GRMF4DYNVMKU2VHQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W0AZNSPW3WX4IL51ARTGAFF2LDSGP42IEHW3EP4CVXYBLCY0
CLIENT_SECRET:PFQMJMGNLQWSXH0SN0ENIQSPO0URKN25GRMF4DYNVMKU2VHQ


## Search for a specific venue category

In [3]:
search_query = 'Mexican'
LIMIT = 500 
cities = ['New York, NY', 'Los Angeles, CA', 'Chicago,IL','Houston, TX','Phoenix, AZ' ]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&query={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET,VERSION, city,search_query,LIMIT)
    results[city] = requests.get(url).json()

In [4]:
results

{'New York, NY': {'meta': {'code': 200,
   'requestId': '5e639ffa1a4b0a001c23aa78'},
  'response': {'venues': [{'id': '4aa44e08f964a5204f4620e3',
     'name': 'Chipotle Mexican Grill',
     'location': {'address': '350 5th Ave Lbby 6',
      'crossStreet': '34th St. between Fifth & Sixth Aves.',
      'lat': 40.748524,
      'lng': -73.9866615,
      'labeledLatLngs': [{'label': 'display',
        'lat': 40.748524,
        'lng': -73.9866615}],
      'postalCode': '10118',
      'cc': 'US',
      'city': 'New York',
      'state': 'NY',
      'country': 'United States',
      'formattedAddress': ['350 5th Ave Lbby 6 (34th St. between Fifth & Sixth Aves.)',
       'New York, NY 10118',
       'United States']},
     'categories': [{'id': '4bf58dd8d48988d1c1941735',
       'name': 'Mexican Restaurant',
       'pluralName': 'Mexican Restaurants',
       'shortName': 'Mexican',
       'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
        'suffix': '.png'},
    

In [108]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [121]:
# transform it into a pandas dataframe
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['venues'])
    df_venues[city] = venues[['name','location.address','location.lat','location.lng', 'location.postalCode']]
    df_venues[city].columns = ['Name', 'Address', 'Latitude', 'Longitude', 'PostalCode']
    df_venues[city]['categories'] = venues.apply(get_category_type, axis=1)

C:\Users\Antonio\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [122]:
df_venues

{'New York, NY':                                       Name  \
 0                   Chipotle Mexican Grill   
 1                   Chipotle Mexican Grill   
 2                   Chipotle Mexican Grill   
 3                   Chipotle Mexican Grill   
 4                   Chipotle Mexican Grill   
 5                   Chipotle Mexican Grill   
 6    Sinigual Contemporary Mexican Cuisine   
 7                   Chipotle Mexican Grill   
 8             Tri Mexican Tacos Food Truck   
 9                   Chipotle Mexican Grill   
 10                  Chipotle Mexican Grill   
 11                  Chipotle Mexican Grill   
 12                  Chipotle Mexican Grill   
 13           Zaragoza Mexican Deli-Grocery   
 14                  Chipotle Mexican Grill   
 15                  Chipotle Mexican Grill   
 16                       Mexican Foodtruck   
 17                  Chipotle Mexican Grill   
 18                  Chipotle Mexican Grill   
 19                  Chipotle Mexican Grill 

## Maps representing mexican food places

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['feature']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['feature']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['feature']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['feature']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  

In [7]:
print('Maps of New York')
maps[cities[0]]

Maps of New York


In [8]:
print('Maps of Los Angeles')
maps[cities[1]]

Maps of Los Angeles


In [9]:
print('Maps of Chicago')
maps[cities[2]]

Maps of Chicago


In [10]:
print('Maps of Houston')
maps[cities[3]]

Maps of Houston


In [11]:
print('Maps of Phoenix')
maps[cities[4]]

Maps of Phoenix


## Average distance of the locations to the mean coordinates.

In [13]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['feature']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['feature']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['feature']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['feature']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.021533484982899625
Los Angeles, CA
Mean Distance from Mean coordinates
0.1054176392208922
Chicago,IL
Mean Distance from Mean coordinates
0.06078129479216396
Houston, TX
Mean Distance from Mean coordinates
0.11810189915281487
Phoenix, AZ
Mean Distance from Mean coordinates
0.10243862120978395


In [15]:
print('Maps of New York')
maps[cities[0]]

Maps of New York


In [16]:
print('Maps of Los Angeles')
maps[cities[1]]

Maps of Los Angeles


In [17]:
print('Maps of Chicago')
maps[cities[2]]

Maps of Chicago


In [18]:
print('Maps of Houston')
maps[cities[3]]

Maps of Houston


In [19]:
print('Maps of Phoenix')
maps[cities[4]]

Maps of Phoenix


## Analysis of each venue category

In [133]:
# We analyzed the categories in New York.
venues_ny = json_normalize(results['New York, NY']['response']['venues'])
newyork_venues = venues_ny[['name','location.address','location.lat','location.lng', 'location.postalCode']]
newyork_venues.columns = ['Name', 'Address', 'Latitude', 'Longitude', 'PostalCode']
newyork_venues['categories'] = venues_ny.apply(get_category_type, axis=1)
newyork_venues

C:\Users\Antonio\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,Address,Latitude,Longitude,PostalCode,categories
0,Chipotle Mexican Grill,350 5th Ave Lbby 6,40.748524,-73.986661,10118,Mexican Restaurant
1,Chipotle Mexican Grill,464 Park Ave S,40.745465,-73.982866,10016,Mexican Restaurant
2,Chipotle Mexican Grill,117 E 14th St,40.734220,-73.988824,10003,Burrito Place
3,Chipotle Mexican Grill,405 Avenue of the Americas,40.733606,-73.999926,10014,Mexican Restaurant
4,Chipotle Mexican Grill,125 E 23rd St Frnt 1,40.739840,-73.985130,10010,Mexican Restaurant
5,Chipotle Mexican Grill,200 Varick St Frnt B,40.728274,-74.005063,10014,Mexican Restaurant
6,Sinigual Contemporary Mexican Cuisine,640 Third Ave,40.750303,-73.975099,10017,Mexican Restaurant
7,Chipotle Mexican Grill,55 E 8th St Frnt 2,40.731162,-73.993437,10003,Mexican Restaurant
8,Tri Mexican Tacos Food Truck,901 Broadway,40.739363,-73.990094,10003,Food Truck
9,Chipotle Mexican Grill,680 Avenue of the Americas,40.741758,-73.993370,10010,Mexican Restaurant


In [169]:
newyork_venues['categories'].unique().tolist()

['Mexican Restaurant',
 'Burrito Place',
 'Food Truck',
 'Gift Shop',
 'Latin American Restaurant',
 'Food']

In [134]:
# Now we are going to group by 'Category' feature:
print('There are {} uniques categories.'.format(len(newyork_venues['categories'].unique())))
newyork_venues.groupby('categories').count()

There are 6 uniques categories.


,Name,Address,Latitude,Longitude,PostalCode
categories,,,,,
Burrito Place,2,2,2,2,2
Food,2,2,2,2,2
Food Truck,5,4,5,5,5
Gift Shop,1,1,1,1,1
Latin American Restaurant,2,2,2,2,2
Mexican Restaurant,38,37,38,38,37


In [138]:
# Transform categorical variable 'Categories' into one hot encoding feature:
newyork_onehot = pd.get_dummies(newyork_venues[['categories']], prefix="", prefix_sep="")

# Add column 'PostalCode' to the new dataframe:
newyork_onehot['PostalCode'] = newyork_venues['PostalCode']

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
manhattan_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()


,Burrito Place,Food,Food Truck,Gift Shop,Latin American Restaurant,Mexican Restaurant,PostalCode
0,0,0,0,0,0,1,10118
1,0,0,0,0,0,1,10016
2,1,0,0,0,0,0,10003
3,0,0,0,0,0,1,10014
4,0,0,0,0,0,1,10010


In [141]:
# Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category
newyork_grouped = newyork_onehot.groupby('PostalCode').mean().reset_index()
newyork_grouped

,PostalCode,Burrito Place,Food,Food Truck,Gift Shop,Latin American Restaurant,Mexican Restaurant
0,07030,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,07302,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,07306,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
3,07310,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,10001,0.000000,0.000000,0.333333,0.000000,0.000000,0.666667
5,10003,0.166667,0.000000,0.166667,0.166667,0.000000,0.500000
6,10004,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
7,10009,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8,10010,0.000000,0.000000,0.250000,0.000000,0.000000,0.750000
9,10011,0.000000,0.666667,0.000000,0.000000,0.000000,0.333333


In [143]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in newyork_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = newyork_grouped[newyork_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----07030----
                venue  freq
0  Mexican Restaurant   1.0
1       Burrito Place   0.0
2                Food   0.0
3          Food Truck   0.0
4           Gift Shop   0.0


----07302----
                venue  freq
0  Mexican Restaurant   1.0
1       Burrito Place   0.0
2                Food   0.0
3          Food Truck   0.0
4           Gift Shop   0.0


----07306----
                       venue  freq
0  Latin American Restaurant   1.0
1              Burrito Place   0.0
2                       Food   0.0
3                 Food Truck   0.0
4                  Gift Shop   0.0


----07310----
                venue  freq
0  Mexican Restaurant   1.0
1       Burrito Place   0.0
2                Food   0.0
3          Food Truck   0.0
4           Gift Shop   0.0


----10001----
                venue  freq
0  Mexican Restaurant  0.67
1          Food Truck  0.33
2       Burrito Place  0.00
3                Food  0.00
4           Gift Shop  0.00


----10003----
                venue  f

In [144]:
# Let's put that into a *pandas* dataframe
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [146]:
# Now let's create the new dataframe and display the top 6 venues for each postal code.
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['PostalCode'] = newyork_grouped['PostalCode']

for ind in np.arange(newyork_grouped.shape[0]):
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,07030,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
1,07302,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
2,07306,Latin American Restaurant,Mexican Restaurant,Gift Shop,Food Truck,Food,Burrito Place
3,07310,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
4,10001,Mexican Restaurant,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place


## Clustering

In [56]:
from sklearn.cluster import KMeans

In [147]:
# set number of clusters 
kclusters = 3

newyork_grouped_clustering = newyork_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 1, 1, 0, 0, 0, 1])

In [148]:
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [151]:
newyork_venues.head()

,Name,Address,Latitude,Longitude,PostalCode,categories
0,Chipotle Mexican Grill,350 5th Ave Lbby 6,40.748524,-73.986661,10118,Mexican Restaurant
1,Chipotle Mexican Grill,464 Park Ave S,40.745465,-73.982866,10016,Mexican Restaurant
2,Chipotle Mexican Grill,117 E 14th St,40.734220,-73.988824,10003,Burrito Place
3,Chipotle Mexican Grill,405 Avenue of the Americas,40.733606,-73.999926,10014,Mexican Restaurant
4,Chipotle Mexican Grill,125 E 23rd St Frnt 1,40.739840,-73.985130,10010,Mexican Restaurant


In [154]:
newyork_venues2 = newyork_venues
newyork_venues2= newyork_venues2.drop(['Name','Address','categories'], axis= 1)

In [156]:
newyork_merged = newyork_venues2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(postalcode_venues_sorted.set_index('PostalCode'), on='PostalCode')

newyork_merged.head() 

,Latitude,Longitude,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,40.748524,-73.986661,10118,0.0,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
1,40.745465,-73.982866,10016,0.0,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
2,40.734220,-73.988824,10003,1.0,Mexican Restaurant,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
3,40.733606,-73.999926,10014,0.0,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
4,40.739840,-73.985130,10010,0.0,Mexican Restaurant,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place


In [160]:
# We deleted the Nan
newyork_merged = newyork_merged.dropna()

In [162]:
def searchCoord(cities):
    address = cities
    #In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent city_explorer 
    geolocator = Nominatim(user_agent="city_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))
    return latitude, longitude
latitude, longitude = searchCoord(cities= 'New York, NY')

The geograpical coordinate of Chicago are 40.7127281, -74.0060152.


In [163]:
# Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['PostalCode'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [ ]:
# Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [164]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,-73.986661,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
1,-73.982866,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
3,-73.999926,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
4,-73.985130,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place
5,-74.005063,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
6,-73.975099,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
9,-73.993370,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place
13,-73.980739,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
14,-73.974354,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place
15,-73.982899,Latin American Restaurant,Gift Shop,Food Truck,Food,Burrito Place


#### Cluster 2

In [165]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
2,-73.988824,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
7,-73.993437,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
8,-73.990094,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
10,-73.996533,Burrito Place,Latin American Restaurant,Gift Shop,Food Truck,Food
11,-73.987871,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place
12,-73.993970,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place
16,-73.992498,Food Truck,Latin American Restaurant,Gift Shop,Food,Burrito Place
17,-73.990074,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
19,-73.993603,Gift Shop,Food Truck,Burrito Place,Latin American Restaurant,Food
25,-74.001218,Mexican Restaurant,Latin American Restaurant,Gift Shop,Food Truck,Burrito Place


#### Cluster 3

In [166]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
44,-74.06213,Mexican Restaurant,Gift Shop,Food Truck,Food,Burrito Place
